In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Email_Spam_Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Email_Spam_Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from emailproject.constants import *
from emailproject.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.multinomialnb
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [8]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        return accuracy


    
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column].values
        
        predicted_qualities = model.predict(test_x)

        accuracy = self.eval_metrics(test_y, predicted_qualities)
        
        # Saving metrics as JSON
        scores = {"accuracy": accuracy}
        self.save_json(path=Path(self.config.metric_file_name), data=scores)
    
    # def save_json(self, path: Path, data: dict):
    #     with open(path, 'w') as f:
    #         json.dump(data, f)


In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-06-09 12:02:15,458: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-09 12:02:15,462: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-09 12:02:15,462: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-09 12:02:15,462: INFO: common: created directory at: artifacts]
[2024-06-09 12:02:15,466: INFO: common: created directory at: artifacts/model_evaluation]


c:\Users\HP\Email_Spam_Prediction\emproj\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MultinomialNB was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: "Subject: eprm 2001 houston  dear speaker ,  i would like to remind you that the room block at the houstonian hotel where  the above mentioned event is being held is about to expire . after friday  20 th april you will not be able to take advantage of the discounted rooms  that are being held there .  please book your accommodation asap to take advantage of this offer .  contact the hotel directly and say that you are part of the risk conference  on 14 & 15 may 2001 . 001 713 680 2626 .  risk waters group do not book accommodation for speakers , you have to  contact them yourself , directly .  if you have not already sent me a short biography and your speaker  checklist , please do so at your earliest convenience .  kind regards  layla o ' leary  event co - ordinator  risk waters group  haymarket house  28 - 29 haymarket  london  swly 4 rx  tel : + 44 ( 0 ) 20 7484 9871  fax : + 44 ( 0 ) 20 7484 9800"